### Running bulk of multimodel testing

This is equivalent to that present in the multimodel wrapper.


In [9]:
from config.read_configurations import config_hbv as hbvArgs
from config.read_configurations import config_prms as prmsArgs
from config.read_configurations import config_sacsma as sacsmaArgs
from config.read_configurations import config_sacsma_snow as sacsmaSnowArgs
from config.read_configurations import config_hbv_hydrodl as hbvhyArgs_d


import torch
import os
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.stats
# from post import plot

from core.utils.randomseed_config import randomseed_config
from core.utils.master import create_output_dirs
from MODELS.loss_functions.get_loss_function import get_lossFun
from MODELS.test_dp_HBV_dynamic import test_dp_hbv
from core.data_processing.data_loading import loadData
from core.data_processing.normalization import transNorm
from core.utils.randomseed_config import randomseed_config
from core.data_processing.model import (
    take_sample_test,
    converting_flow_from_ft3_per_sec_to_mm_per_day
)

import warnings
warnings.filterwarnings("ignore")



# Set path to `hydro_multimodel_results` directory.
if platform.system() == 'Darwin':
    # For mac os
    out_dir = '/Users/leoglonz/Desktop/water/data/model_runs/hydro_multimodel_results'
    # Some operations are not yet working with MPS, so we might need to set some environment variables to use CPU fall instead
    # %env PYTORCH_ENABLE_MPS_FALLBACK=1

elif platform.system() == 'Windows':
    # For windows
    out_dir = 'D:\\data\\model_runs\\hydro_multimodel_results\\'

elif platform.system() == 'Linux':
    # For Colab
    out_dir = '/content/drive/MyDrive/Colab/data/model_runs/hydro_multimodel_results'

else:
    raise ValueError('Unsupported operating system.')


##-----## Multi-model Parameters ##-----##
##--------------------------------------##
# Setting dictionaries to separately manage each diff model's attributes.
models = {'dPLHBV_dyn': None, 'SACSMA_snow':None, 'marrmot_PRMS':None}  # 'HBV':None, 'hbvhy': None, 'SACSMA_snow':None, 'SACSMA':None,
args_list = {'dPLHBV_dyn': hbvhyArgs_d, 'SACSMA_snow':sacsmaSnowArgs, 'marrmot_PRMS':prmsArgs}   # 'hbvhy': hbvhyArgs, 'HBV' : hbvArgs, 'SACSMA_snow':None, 'SACSMA': sacsmaArgs,
ENSEMBLE_TYPE = 'max'  # 'median', 'avg', 'max', 'softmax'

# Load test observations and predictions from a prior run.
pred_path = os.path.join(out_dir, 'multimodels', '671_sites_dp', 'output', 'preds_671_dPLHBVd_SACSMASnow_PRMS.npy')
obs_path = os.path.join(out_dir, 'multimodels', '671_sites_dp', 'output', 'obs_671_dPLHBVd_SACSMASnow_PRMS.npy')
preds = np.load(pred_path, allow_pickle=True).item()
obs = np.load(obs_path, allow_pickle=True).item()

model_output = preds
y_obs = obs

loading package hydroDL


In [50]:
sac = y_obs['SACSMA_snow'].squeeze().numpy()
per = y_obs['marrmot_PRMS'].squeeze().numpy()

per.shape

(5114, 671)

In [51]:
np.array_equal(sac,per, equal_nan=True)


True

In [10]:
def calFDC(data):
    # data = Ngrid * Nday
    Ngrid, Nday = data.shape
    FDC100 = np.full([Ngrid, 100], np.nan)
    for ii in range(Ngrid):
        tempdata0 = data[ii, :]
        tempdata = tempdata0[~np.isnan(tempdata0)]
        # deal with no data case for some gages
        if len(tempdata)==0:
            tempdata = np.full(Nday, 0)
        # sort from large to small
        temp_sort = np.sort(tempdata)[::-1]
        # select 100 quantile points
        Nlen = len(tempdata)
        ind = (np.arange(100)/100*Nlen).astype(int)
        FDCflow = temp_sort[ind]
        if len(FDCflow) != 100:
            raise Exception('unknown assimilation variable')
        else:
            FDC100[ii, :] = FDCflow

    return FDC100


def statError(pred, target):
    ngrid, nt = pred.shape
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
    # Bias
        Bias = np.nanmean(pred - target, axis=1)
        # RMSE
        RMSE = np.sqrt(np.nanmean((pred - target)**2, axis=1))
        # ubRMSE
        predMean = np.tile(np.nanmean(pred, axis=1), (nt, 1)).transpose()
        targetMean = np.tile(np.nanmean(target, axis=1), (nt, 1)).transpose()
        predAnom = pred - predMean
        targetAnom = target - targetMean
        ubRMSE = np.sqrt(np.nanmean((predAnom - targetAnom)**2, axis=1))
        # FDC metric
        predFDC = calFDC(pred)
        targetFDC = calFDC(target)
        FDCRMSE = np.sqrt(np.nanmean((predFDC - targetFDC) ** 2, axis=1))
    # rho R2 NSE
        Corr = np.full(ngrid, np.nan)
        CorrSp = np.full(ngrid, np.nan)
        R2 = np.full(ngrid, np.nan)
        NSE = np.full(ngrid, np.nan)
        PBiaslow = np.full(ngrid, np.nan)
        PBiashigh = np.full(ngrid, np.nan)
        PBias = np.full(ngrid, np.nan)
        PBiasother = np.full(ngrid, np.nan)
        KGE = np.full(ngrid, np.nan)
        KGE12 = np.full(ngrid, np.nan)
        RMSElow = np.full(ngrid, np.nan)
        RMSEhigh = np.full(ngrid, np.nan)
        RMSEother = np.full(ngrid, np.nan)
        for k in range(0, ngrid):
            x = pred[k, :]
            y = target[k, :]
            ind = np.where(np.logical_and(~np.isnan(x), ~np.isnan(y)))[0]
            if ind.shape[0] > 0:
                xx = x[ind]
                yy = y[ind]
                # percent bias
                PBias[k] = np.sum(xx - yy) / np.sum(yy) * 100

                # FHV the peak flows bias 2%
                # FLV the low flows bias bottom 30%, log space
                pred_sort = np.sort(xx)
                target_sort = np.sort(yy)
                indexlow = round(0.3 * len(pred_sort))
                indexhigh = round(0.98 * len(pred_sort))
                lowpred = pred_sort[:indexlow]
                highpred = pred_sort[indexhigh:]
                otherpred = pred_sort[indexlow:indexhigh]
                lowtarget = target_sort[:indexlow]
                hightarget = target_sort[indexhigh:]
                othertarget = target_sort[indexlow:indexhigh]
                PBiaslow[k] = np.sum(lowpred - lowtarget) / np.sum(lowtarget) * 100
                PBiashigh[k] = np.sum(highpred - hightarget) / np.sum(hightarget) * 100
                PBiasother[k] = np.sum(otherpred - othertarget) / np.sum(othertarget) * 100
                RMSElow[k] = np.sqrt(np.nanmean((lowpred - lowtarget)**2))
                RMSEhigh[k] = np.sqrt(np.nanmean((highpred - hightarget)**2))
                RMSEother[k] = np.sqrt(np.nanmean((otherpred - othertarget)**2))

                if ind.shape[0] > 1:
                    # Theoretically at least two points for correlation
                    Corr[k] = scipy.stats.pearsonr(xx, yy)[0]
                    CorrSp[k] = scipy.stats.spearmanr(xx, yy)[0]
                    yymean = yy.mean()
                    yystd = np.std(yy)
                    xxmean = xx.mean()
                    xxstd = np.std(xx)
                    KGE[k] = 1 - np.sqrt((Corr[k]-1)**2 + (xxstd/yystd-1)**2 + (xxmean/yymean-1)**2)
                    KGE12[k] = 1 - np.sqrt((Corr[k] - 1) ** 2 + ((xxstd*yymean)/ (yystd*xxmean) - 1) ** 2 + (xxmean / yymean - 1) ** 2)
                    SST = np.sum((yy-yymean)**2)
                    SSReg = np.sum((xx-yymean)**2)
                    SSRes = np.sum((yy-xx)**2)
                    R2[k] = 1-SSRes/SST
                    NSE[k] = 1-SSRes/SST

    outDict = dict(Bias=Bias, RMSE=RMSE, ubRMSE=ubRMSE, Corr=Corr, CorrSp=CorrSp, R2=R2, NSE=NSE,
                   FLV=PBiaslow, FHV=PBiashigh, PBias=PBias, PBiasother=PBiasother, KGE=KGE, KGE12=KGE12, fdcRMSE=FDCRMSE,
                   lowRMSE=RMSElow, highRMSE=RMSEhigh, midRMSE=RMSEother)

    return outDict

In [2]:
class hydroEnsemble(torch.nn.Module):
    # Wrapper for multiple hydrologic models.
    # In future, consider just passing the models you want to ensemble explicitly.
    def __init__(self, num_models, hidden_size, num_layers):
        super(hydroEnsemble, self).__init__()

        self.lstm = torch.nn.LSTM(num_models, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_models)  # Two models (modelA and modelB)

        # self.modelA = modelA
        # self.modelB = modelB
        # self.classifier = torch.nn.Linear(4, 2)

    def forward(self, x):
        # x is the input sequence tensor with shape (batch_size, sequence_length, num_models)

        # Setting randomseed for deterministic output.
        randomseed_config(0)

        # Add batch dimension to input and convert to tensor.
        x_exp = x.unsqueeze(0)

        # LSTM layer
        lstm_out, _ = self.lstm(x_exp)

        # Fully connected layer
        fc_out = self.fc(lstm_out)

        # Apply softmax activation to obtain weights
        weights = torch.nn.functional.softmax(fc_out, dim=2).squeeze()

        # Weighted combination of predictions.
        weighted_preds = np.multiply(weights.detach(), x)

        # Or take the max weight and return the corresponding value.
        max_vals, _ = torch.max(weights, dim=1)
        btensor = torch.zeros_like(weights)
        btensor[weights==max_vals.view(-1,1)] = 1
        weighted_preds = np.multiply(btensor.detach(), x)

        preds = torch.sum(weighted_preds, dim=1)

        # All tensors
        # return preds, weights, weighted_preds
        return preds


NameError: name 'torch' is not defined

In [378]:
def calculate_metrics_multi(args_list, model_outputs, y_obs_list, ensemble_type='max', out_dir=None):
    """
    Calculate stats for a multimodel ensemble.
    """
    stats_list = dict()

    for mod in args_list:
        args = args_list[mod]
        mod_out = model_outputs[mod]
        y_obs = y_obs_list[mod]

        if mod in ['SACSMA', 'SACSMA_snow', 'marrmot_PRMS', 'HBV']:
            # Note for hydrodl HBV, calculations have already been done so skip.

            # Saving data
            if out_dir:
                path = os.path.join(out_dir, 'models', '671_sites_dp', mod)
                if not os.path.exists(path):
                    os.makedirs(path, exist_ok=True)

                # Test data (obs and model results).
                for key in mod_out[0].keys():
                    if len(mod_out[0][key].shape) == 3:
                        dim = 1
                    else:
                        dim = 0
                    concatenated_tensor = torch.cat([d[key] for d in mod_out], dim=dim)
                    file_name = key + ".npy"
                    np.save(os.path.join(path, file_name), concatenated_tensor.numpy())
                    # np.save(os.path.join(args["out_dir"], args["testing_dir"], file_name), concatenated_tensor.numpy())

                # Reading and flow observations.
                print(args['target'])
                for var in args["target"]:
                    item_obs = y_obs[:, :, args["target"].index(var)]
                    file_name = var + ".npy"
                    np.save(os.path.join(path, file_name), item_obs)
                    # np.save(os.path.join(args["out_dir"], args["testing_dir"], file_name), item_obs)


            ###################### calculations here ######################
            pred_list = list()
            obs_list = list()
            flow_sim = torch.cat([d["flow_sim"] for d in mod_out], dim=1)
            flow_obs = y_obs[:, :, args["target"].index("00060_Mean")]

            pred_list.append(flow_sim.numpy())
            obs_list.append(np.expand_dims(flow_obs, 2))
            

            # we need to swap axes here to have [basin, days], and remove redundant
            # dimensions with np.squeeze().
            stats_list[mod] = [
                statError(np.swapaxes(x.squeeze(), 1, 0), np.swapaxes(y.squeeze(), 1, 0))
                for (x, y) in zip(pred_list, obs_list)
            ]
        elif mod in ['hbvhy', 'dPLHBV_dyn']:
            stats_list[mod] = [statError(mod_out[:,:,0], y_obs.squeeze())]
        else:
            raise ValueError(f"Unsupported model type in `models`.")

    # Calculating final statistics for the whole set of basins.
    name_list = ["flow"]
    for st, name in zip(stats_list[mod], name_list):
        count = 0
        mdstd = np.zeros([len(st), 3])
        for key in st.keys():
            # st contains the statistics on a model run like NSE and KGE.
            # Find the best result (e.g., the max, avg, median) and merge from each model.
            for i, mod in enumerate(args_list):
                if i == 0:
                    # temp contains the values of key per basin.
                    temp = stats_list[mod][0][key]
                    continue
                elif i == 1:
                    temp = np.stack((temp, stats_list[mod][0][key]), axis=1)
                else:
                    temp = np.hstack((temp, stats_list[mod][0][key].reshape(-1,1)))

            print(temp, temp.shape)
            
            if len(args_list) > 1:
                if ensemble_type == 'max':
                    # print(temp, key)
                    temp = np.amax(temp, axis=1)
                    # print(temp, key)
                elif ensemble_type == 'avg':
                    temp = np.mean(temp, axis=1)
                elif ensemble_type == 'median':
                    temp = np.median(temp, axis=1)
                elif ensemble_type == 'softmax':
                    # # Softmax gets relative contributions of each model.
                    # weights = torch.nn.functional.softmax(torch.from_numpy(temp), dim=1)
                    # temp = np.sum(temp * weights.numpy(), axis=1)

                    # Instantiate weighting lstm with softmax.
                    lstm = hydroEnsemble(num_models=len(args_list), hidden_size=192, num_layers=3)
                    # Forward pass through the model
                    temp = lstm(torch.tensor(temp, dtype=torch.float))
                else:
                    raise ValueError("Invalid model ensemble type specified.")

            median = np.nanmedian(temp)  # abs(i)
            std = np.nanstd(temp)  # abs(i)
            mean = np.nanmean(temp)  # abs(i)
            k = np.array([[median, std, mean]])
            mdstd[count] = k
            count = count + 1

        # mdstd displays the statistics for each error measure in stats_list.
        mdstd = pd.DataFrame(
            mdstd, index=st.keys(), columns=["median", "STD", "mean"]
        )
        # Save the data stats from the training run:
        if out_dir and len(args_list) > 1:
            path = os.path.join(out_dir, 'multimodels', '671_sites_dp', 'n_' + ensemble_type)
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)

            mdstd.to_csv((os.path.join(path, "mdstd_" + name + "_" + ensemble_type +".csv")))
        elif out_dir:
            path = os.path.join(out_dir, 'models', '671_sites_dp', args_list[0])
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)

            mdstd.to_csv((os.path.join(path, "mdstd_" + name + "_" + ".csv")))
        else: continue

    # Show boxplots of the results
    # plt.rcParams["font.size"] = 14
    # keyLst = ["Bias", "RMSE", "ubRMSE", "NSE", "Corr"]
    # dataBox = list()
    # for iS in range(len(keyLst)):
    #     statStr = keyLst[iS]
    #     temp = list()
    #     # for k in range(len(st)):
    #     data = st[statStr]
    #     data = data[~np.isnan(data)]
    #     temp.append(data)
    #     dataBox.append(temp)
    # labelname = [
    #     "Hybrid differentiable model"
    # ]  # ['STA:316,batch158', 'STA:156,batch156', 'STA:1032,batch516']   # ['LSTM-34 Basin']

    # xlabel = ["Bias ($\mathregular{deg}$C)", "RMSE", "ubRMSE", "NSE", "Corr"]
    # fig = plot.plotBoxFig(
    #     dataBox, xlabel, label2=labelname, sharey=False, figsize=(16, 8)
    # )
    # fig.patch.set_facecolor("white")
    # boxPlotName = "PGML"
    # fig.suptitle(boxPlotName, fontsize=12)
    # plt.rcParams["font.size"] = 12
    # # plt.savefig(
    # #     os.path.join(args["out_dir"], args["testing_dir"], "Box_" + name + ".png")
    # # )  # , dpi=500
    # # fig.show()
    # plt.close()

    torch.cuda.empty_cache()
    print("Testing ended")

    return stats_list, mdstd


In [93]:
args_list = {'dPLHBV_dyn': hbvhyArgs_d, 'SACSMA_snow':sacsmaSnowArgs, 'marrmot_PRMS':prmsArgs}

# Initialize
flow_preds = []
flow_obs = None
obs_trig = False

# Concatenate individual model predictions, and observation data.
for i, mod in enumerate(args_list):
    args = args_list[mod]
    mod_out = model_output[mod]
    y_ob = y_obs[mod]

    if mod in ['HBV', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']:
        # Hydro models are tested in batches, so we concatenate them and select
        # the desired flow.
        # Note: modified HBV already has this preparation done during testing.

        # Get flow predictions and swap axes to get shape [basins, days]
        pred = np.swapaxes(torch.cat([d["flow_sim"] for d in mod_out], dim=1).squeeze().numpy(), 0, 1)

        if obs_trig == False:
            # dPLHBV uses GAGES while the other hydro models use CAMELS data. This means small 
            # e-5 variation in observation data between the two. This is averaged if both models
            # are used, but to avoid double-counting data from multiply hydro models, use a trigger.
            obs = np.swapaxes(y_ob[:, :, args["target"].index("00060_Mean")].numpy(), 0, 1)
            obs_trig = True
            dup = False
        else:
            dup = True
            

    elif mod in ['dPLHBV_dyn']:
        pred = mod_out[:,:,0][:,365:] # Set dim2 = 0 to get streamflow Qr
        obs = y_ob.squeeze()[:,365:]
        dup = False

    else:
        raise ValueError(f"Unsupported model type in `models`.")
    
    if i == 0:
        tmp_pred = pred
        tmp_obs = obs
    elif i == 1:
        tmp_pred = np.stack((tmp_pred, pred), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.stack((tmp_obs, obs), axis=2)
    else:
        # Combine outputs of >3 models.
        tmp_pred = np.concatenate((tmp_pred,np.expand_dims(pred, 2)), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.concatenate((tmp_obs,np.expand_dims(obs, 2)), axis=2)



In [94]:
# models = {'SACSMA':None, 'marrmot_PRMS':None}  # 'HBV':None
args_list = {'dPLHBV_dyn': hbvhyArgs_d,'SACSMA_snow':sacsmaSnowArgs}   # 'hbvhy': hbvhyArgs, 'HBV' : hbvArgs, 'SACSMA_snow':None, 'SACSMA': sacsmaArgs,
ENSEMBLE_TYPE = 'avg'  # 'median', 'avg', 'max', 'softmax'
stats_list, mtstd = calculate_metrics_multi(args_list, model_outputs=model_output, y_obs_list=y_obs, ensemble_type=ENSEMBLE_TYPE)

mtstd['median']

NameError: name 'calculate_metrics_multi' is not defined